# imgAna_OpenCV 

In the past a service was invoked to locate faces, this is a replacement for that outboard processing locating
the faces using an OpenCV components.
This works, it is still in process.

## What is going on here

The first part of the file is getting access to OpenCV, verify that it works in notebook and possibly in Streams. 
This work is done working with face detection.  A good part of this work is dealing with renderng the images 
and adding boxes to regions, if your going todo something with images you gotta show them. 


After we work out that it can be used I work out how to fetch images from the web, using the url in the
tuple. The image must be converted into somthing that JSON will not be upset with. 

Two types of analysis are being done on the images. 
- FaceRegion() : finds faces, this what I used to learn on, thus all the work a the begining of the notebook.
- ObjectRegions() : finds objects, much better example since it give the location of the object and an key to what the object is. This I got from Jerome. 

Two applications in here are build here. 
- imageFetch : pulls images in from the web and encodes them
- fineImageElements : invokes FaceRegions() to find faces & ObjectRegions()

The 'imageFetch' application gets it's data from [WikiRecentPhase3](./imgAna_3.jupyter-py36.ipynb), 
make sure you read about this below. 





In [ ]:
#!pip install opencv-contrib-python
#%load_ext autoreload
#%autoreload 2

## Note : 

Getting this build locally and push up to the cloud you need the following...
```bash
pip install opencv-contrib-python
pip install pandas
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
pip install beautifulsoup4
pip install Pillow
```
Here is where I got a jump start on [face-detection-python-opencv](https://www.datacamp.com/community/tutorials/face-detection-python-opencv). 


In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd

from IPython.core.debugger import set_trace
from IPython.display import display, clear_output

from statistics import mean
from collections import deque
from collections import Counter

import json
import datetime 

import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout

from bs4 import BeautifulSoup

%matplotlib inline

from sseclient import SSEClient as EventSource

from ipywidgets import Button, HBox, VBox, Layout

from  functools import lru_cache
import requests

from streamsx.topology.topology import *
import streamsx.rest as rest
from streamsx.topology import context

import time

from PIL import Image,  ImageDraw  # https://pillow.readthedocs.io/en/4.3.x/
import io
import base64
import sys
if '../scripts' not in sys.path:
    sys.path.insert(0, '../scripts')
import cvsupport
import streams_operations
import streams_render
import streamsx.eventstreams as eventstreams
from streamsx.topology.schema import CommonSchema

from IPython.core.display import HTML

## has credentials for cloud & kafka/event stream
import credential


import logging
logging.getLogger("imgAna_OpenCV")



In [ ]:
## HACK - his should note be necessary - something is wrong with my environment, 
##
import os
try:
    print("JAVA_HOME is set to",os.environ["JAVA_HOME"])
except KeyError as err:
    os.environ["JAVA_HOME"] = "/Library/Java/JavaVirtualMachines/jdk1.8.0_221.jdk/Contents/Home"
    print("Had to set JAVA_HOME is set to",os.environ["JAVA_HOME"],"\n   --- what is going on here...")


In [ ]:
def catchInterrupt(func):
    """decorator : when interupt occurs the display is lost if you don't catch it
       TODO * <view>.stop_data_fetch()  # stop
       
    """
    def catch_interrupt(*args, **kwargs):
        try: 
            func(*args, **kwargs)
        except (KeyboardInterrupt): pass
    return catch_interrupt

## Process a local file
Using OpenCv so some  processing on a local file.
* do some image manipulation
* find the face 

In [ ]:
img_raw = cv2.imread('../datasets/baby1.png')
print(type(img_raw))
print(img_raw.shape)
plt.imshow(img_raw)

In [ ]:
def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(convertToRGB(img_raw))

In [ ]:
def convertToBW(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
plt.imshow(convertToBW(img_raw), cmap='gray')

In [ ]:
haar_cascade_face = cv2.CascadeClassifier('../datasets/haarcascade_frontalface_default.xml')
haar_cascade_face

faces_rects = haar_cascade_face.detectMultiScale(convertToBW(img_raw), scaleFactor = 1.2, minNeighbors = 5);

# Let us print the no. of faces found
print("Faces found: {} faces within regions:{}".format(len(faces_rects), faces_rects))


## Visualize 
* Find faces
* Draw a box around them

In [ ]:
def detect_faces(image, cascade, scaleFactor = 1.1):
    # create a copy of the image to prevent any changes to the original one.
    image_copy = image.copy()

    #convert the test image to gray scale as opencv face detector expects gray images
    gray_image = convertToBW(image_copy)

    # Applying the haar classifier to detect faces
    image_rects = cascade.detectMultiScale(gray_image, scaleFactor=scaleFactor, minNeighbors=5)
    return image_rects

def rects_render(image, rects):
    image_copy = image.copy()
    for (x, y, w, h) in rects:
        cv2.rectangle(image_copy, (x, y), (x+w, y+h), (0, 255, 0), 15)
    return image_copy
    
## Render the image with the faces_rects
if len(faces_rects)> 0:
    img_rects = rects_render(convertToRGB(img_raw), faces_rects)
    #print(img_rects)
    plt.imshow(img_rects)
else:
    print("On image")

In [ ]:
## Another image....
img_raw = cv2.imread('../datasets/imgClassify.jpg')
img_rgb = convertToRGB(img_raw)
rects = detect_faces(img_rgb, haar_cascade_face)
img_rects = rects_render(img_rgb, rects)
plt.imshow(img_rects)

# Fetch a image from the web


Use the the URLs for testing.

In [ ]:
# Get and image from the web. 
urls = ["https://media.wired.com/photos/5c5354d391d0df22c1dee493/master/w_2560%2Cc_limit/Backchannel-Lena-Final.jpg",
       "https://upload.wikimedia.org/wikipedia/commons/1/1e/Christopher_de_Paus.JPG",
       "https://upload.wikimedia.org/wikipedia/commons/a/a5/Anne_Bethel_Spencer_in_her_wedding_dress.jpg",
       "https://upload.wikimedia.org/wikipedia/commons/f/f7/Barbara_Anderson_1969.JPG"]
response = requests.get(urls[0])

### OpenCV locate face region : face_rects
- OpenCV to fetch process image, locate the faces.
- OpenCV to render the results.

This processing will be pushed to Streams.


#### face_rects[] is a list of retanges that the face is in

In [ ]:
def bts_to_img(bts):
    '''
    :param bts: results from image_to_bts
    '''
    buff = np.fromstring(bts, np.uint8)
    buff = buff.reshape(1, -1)
    img = cv2.imdecode(buff, cv2.IMREAD_COLOR)
    return img

img_raw = bts_to_img(response.content)
print("Size of image to process : ",img_raw.shape)
img_rgb = convertToRGB(img_raw)
face_rects = detect_faces(img_rgb, haar_cascade_face)
print("Found: {} potential faces. \n {}".format(len(face_rects),  face_rects))
img_rects = rects_render(img_rgb, face_rects)
plt.imshow(img_rects)

## Render with face_rects[] - reduce dependency on numpy
Using image from web add the rects. 

- Render in a widget, in a browser. 



In [ ]:
def line_box(ele):
    (x,y,w,h)=ele
    return (x,y, x+w,y, x+w,y+h, x,y+h, x,y)
    

def inscribe_rect(bin_image, detection_box, box_line_width=10):
    """Inscribe box on image
    
    This is updating the image passed in.
    
    Args:
        bin_image : binary image
        detection_box : region to put box around
    Return:
        return image - 
    """
    draw = ImageDraw.Draw(bin_image) 
    draw.line(line_box(detection_box), fill="yellow", width=box_line_width)
    return bin_image
   

def encode_img(img):
    """must be easier way"""
    with io.BytesIO() as output:
        img.save(output, format="JPEG")
        contents = output.getvalue() 
    return base64.b64encode(contents).decode('ascii')

def decode_img(bin64):
    """must be easier way"""
    img = Image.open(io.BytesIO(base64.b64decode(bin64)))
    return img

def resize_image(bin_image, basewidth=None, baseheight=None):
    """Resize image proportional to the base, make it fit in cell"""
    if basewidth is not None:
        wpercent = (basewidth/float(bin_image.size[0]))
        hsize = int((float(bin_image.size[1])*float(wpercent)))
        return bin_image.resize((basewidth,hsize), Image.ANTIALIAS)
    wpercent = (baseheight/float(bin_image.size[1]))
    wsize = int((float(bin_image.size[0])*float(wpercent)))
    return bin_image.resize((wsize,baseheight), Image.ANTIALIAS)

In [ ]:
## Widget to display image
demo_widget = widgets.Output(layout={'border': '1px solid green'})
display(demo_widget)
with Image.open(io.BytesIO(response.content)) as bin_image:
    for rect in face_rects:
        inscribe_rect(bin_image, rect)
    with demo_widget:
        print(type(bin_image))
        display(bin_image)
        clear_output(wait=True)

## Get images from Wiki via Streams

At this point...
- we can fetch images local/web
- render the image in the browser
- locate faces using the OpenCV facility.
- render the image inscribed with 'suspected' face regions. 

Before we push this processing up to Streams we'll process the types of images the Streams applicagtion will be dealing with locally. 
If you have not you need to bring up the the Streams application composed in the [WikiRecentPhase3](./imgAna_3.jupyter-py36.ipynb) notebook. 
This application's view (soupActive) includes the URL of the image being submitted to WikiPedia, the image that we 
are going preform face OpenCV face detection.

We'll do the processing on the Streams, results will be send back. Below were going 
the processing in the browser to work out the processing to do. 

This assumes that you have started the [WikiRecentPhase3](./imgAna_3.jupyter-py36.ipynb) 



<a name="setup"></a>
# Setup
### Add credentials for the IBM Streams service

#### ICPD setup

With the cell below selected, click the "Connect to instance" button in the toolbar to insert the credentials for the service.

<a target="blank" href="https://developer.ibm.com/streamsdev/wp-content/uploads/sites/15/2019/02/connect_icp4d.gif">See an example</a>.

## Setup environment to submit and view.
- Use the data from WikiRecentPhase3

In [ ]:
%%script false --no-raise-error

# If you using ICP4D insert your creds here...
#from icpd_core import icpd_util
#global cfg
#cfg=icpd_util.get_service_instance_details(name='zen-sample-icp1-blitz-env')

# Accessing Streams Instance & views

instance,cfg = streams_operations.get_instance(service_name='Streaming3Turbine')


## Fetch the view that has the image URLs.
_view = instance.get_views(name="soupActive")[0]
_view.start_data_fetch()

In [ ]:
def commonSubmit(_cfg, streams_cloud_service, _topo):
    """submit to either the cloud or CP4D
    Args:
        _cfg: when submitting from CP4D.
        streams_cloud_service : when submitting to cloud, the name of the service credential.py must appropriate mapping
        _topo : topology to submit
    
    """
    if _cfg is not None:
        # Disable SSL certificate verification if necessary
        _cfg[context.ConfigParams.SSL_VERIFY] = False
        submission_result = context.submit("DISTRIBUTED",_topo, config=_cfg)
    if _cfg is None:
        cloud = {
            context.ConfigParams.VCAP_SERVICES: credential.vcap_conf,
            context.ConfigParams.SERVICE_NAME: streams_cloud_service,
            context.ContextTypes.STREAMING_ANALYTICS_SERVICE:"STREAMING_ANALYTIC",
            context.ConfigParams.FORCE_REMOTE_BUILD: True,
        }
        submission_result = context.submit("STREAMING_ANALYTICS_SERVICE",_topo,config=cloud)

    # The submission_result object contains information about the running application, or job
    if submission_result.job:
        report = "JobId:{} Name:{} ".format(submission_result['id'], submission_result['name'])
    else:
        report = "Somthing did work:{}".format(submission_result)


In [ ]:
def display_image(view_tuple, image_region=None, rect_region=None, title_region=None, status_region=None, url_region=None):
    img_url = view_tuple['img_desc'][0]['img']
    response = requests.get(img_url)
    
    img_raw = bts_to_img(response.content)
    if img_raw is None:
        print("img_url:{} bts_img() failed".format(img_url))
        return
    img_rgb = convertToRGB(img_raw)
    face_rects = detect_faces(img_rgb, haar_cascade_face)
    status_region.value = "Found: {} potential faces. \n {}".format(len(face_rects),  face_rects)
    # img_rects = rects_render(img_rgb, face_rects)
    url_region.value = img_url
    title_region.value = view_tuple['title']
    with Image.open(io.BytesIO(response.content)) as bin_image:
        with image_region:
            display(resize_image(bin_image, baseheight=300))
            clear_output(wait=True)
            for rect in face_rects:
                inscribe_rect(bin_image, rect)
            with rect_region:
                if len(face_rects) is not 0:
                    display(resize_image(bin_image, baseheight=300))
                    clear_output(wait=True)
                else:
                    clear_output()
         

# Render in 'Dashboard'

In [ ]:
%%script false --no-raise-error

## Render in 'Dashboard' images that processed locally using OpenCV. Best to get things working 

## locally before pushing it up to Streams. 
## 
## Images are coming from a Wikipedia via a Streams view.
##
status_local = widgets.Label(value="Status", layout={'border': '1px solid green','width':'600%'})
url_local = widgets.Label(value="Img URL", layout={'border': '1px solid green','width':'60%'})
image_local = widgets.Output(layout={'border': '1px solid red','width':'30%','height':'300pt'})
rect_local = widgets.Output(layout={'border': '1px solid red','width':'30%','height':'300pt'})
hbox_local = widgets.HBox([image_local, rect_local])
title_local = widgets.Label(value="Title", layout={'border': '1px solid green','width':'60%'})
dashboard = widgets.VBox([status_local, hbox_local, title_local, url_local])
display(dashboard)

In [ ]:
%%script false --no-raise-error

# This cell is going to populate the dashboard above.
## The left pane has the image that arrived, the right 
## show the faces demarked, if none found no image.
## 
## - Re-execute the cell to fetch/process/render images.
## - Note the debug statements below.

@catchInterrupt
def server_soup(count=1):
    """Fetch and display images from view.
    Args::
        count: number of iterations to fetch images, count<0
        is infinite
    """
    while count != 0:
        count -= 1
        view_tuples = _view.fetch_tuples(max_tuples=12, timeout=2)
        print("remaining cycles:{} tuples:{}".format(count, len(view_tuples)))
        for soup_tuple in view_tuples:
            display_image(soup_tuple, image_region=image_local, rect_region=rect_local, title_region=title_local, status_region=status_local, url_region=url_local)

server_soup()

## Streams processing


##   To Processing Operators. 
Above we created code to fetch a image from the web and find faces, which is composed a number of functions. 
To push it up to Streams we've rolled up the processing into two functions.

The cvsupport.py in [scripts](../scripts) class that are used below to compose the application.
- ImageFetch[] - web request that puts image in tuple. The most time consuming portion of the processing is fetching the images using the URL. Retrieve the image from and push the image data into the Stream where it will be processed by downstream operator.
- FaceRegions[] - locates faces in image 

### Test the ImageFetch[] & FaceRegions[] before submitting ImageAnalysis application

In [ ]:
url = "https://media.wired.com/photos/5c5354d391d0df22c1dee493/master/w_2560%2Cc_limit/Backchannel-Lena-Final.jpg"
#url = "https://upload.wikimedia.org/wikipedia/commons/0/0a/Great_Western_Road%2C_53-100_Kelvin_Court%2C_Glasgow.jpg"
tuple1 = {'img_desc':[{'img':url}]}   # simulate the tuple to process
image_fetch = cvsupport.ImageFetch()

# simulate the Stream.
tuple2 = image_fetch(tuple1)
with cvsupport.FaceRegions(haar_file="../datasets/haarcascade_frontalface_default.xml") as fr:
    print(type(fr))
    tuple3 = fr(tuple2)
#face_region = cvsupport.FaceRegions(haar_file="../datasets/haarcascade_frontalface_default.xml")
#tuple3 = face_region(tuple2)

# render the results.... create a
if tuple3 is not None:
    print(" URL:{}\n had {} regions \n ... {}".format(tuple3['img_desc'][0]['img'], len(tuple3['face_regions']), tuple3['face_regions']))
    verify_widget = widgets.Output(layout={'border': '1px solid green'})
    display(verify_widget)

    response = requests.get(tuple3['img_desc'][0]['img'])  # fetch the image

    with Image.open(io.BytesIO(response.content)) as bin_image:
        for rect in tuple3['face_regions']:
            inscribe_rect(bin_image, rect)
        with verify_widget:
            display(resize_image(bin_image, baseheight=600))
else:
    print("No faces located in image....")

## Test ObjectDetect locally before pushing up

In [ ]:
#url = "https://upload.wikimedia.org/wikipedia/commons/0/0a/Great_Western_Road%2C_53-100_Kelvin_Court%2C_Glasgow.jpg"
urls = ["https://upload.wikimedia.org/wikipedia/commons/1/1e/Christopher_de_Paus.JPG",
"https://upload.wikimedia.org/wikipedia/commons/a/a5/Anne_Bethel_Spencer_in_her_wedding_dress.jpg",
"https://upload.wikimedia.org/wikipedia/commons/f/f7/Barbara_Anderson_1969.JPG"]
tuple1 = {'img_desc':[{'img':urls[2]}]}   # simulate the tuple to process
image_fetch = cvsupport.ImageFetch()

# simulate the Stream.
tuple2 = image_fetch(tuple1)


with cvsupport.ObjectRegions(classes="../datasets/yolov3.txt", 
                             weights="../datasets/yolov3.weights", 
                             config="../datasets/yolov3.cfg",
                             on_streams=False) as fr:
    print(type(fr))
    tuple3 = fr(tuple2)
#face_region = cvsupport.FaceRegions(haar_file="../datasets/haarcascade_frontalface_default.xml")
#tuple3 = face_region(tuple2)

# render the results.... create a
if tuple3 is not None:
    print(" URL:{}\n had {} regions \n ... {}".format(tuple3['img_desc'][0]['img'], len(tuple3['object_regions']), tuple3['object_regions']))
    verify_widget = widgets.Output(layout={'border': '1px solid green'})
    display(verify_widget)

    response = requests.get(tuple3['img_desc'][0]['img'])  # fetch the image

    with Image.open(io.BytesIO(response.content)) as bin_image:
        for obj in tuple3['object_regions']:
            inscribe_rect(bin_image, obj["region"])
        with verify_widget:
            display(resize_image(bin_image, baseheight=600))
else:
    print("No objects located in image....")

In [ ]:
print(tuple3.keys())
print(tuple3['object_regions'][0])
type(tuple3['object_regions'][0]['confidence'])


# Compose and Submit the applications.

The Streams application that will get Tuples by subscribing a feed
that is composed in the [WikiRecentPhase3](./imgAna_3.jupyter-py36.ipynb).

### what applications are currently running....
Cancel those that you are about to submit...

In [ ]:
streams_render.list_jobs(instance, cancel=True)

### What views are available?
At the start only WikiPhase3


In [ ]:
streams_render.display_views(instance, "WikiPhase3")
streams_render.display_views(instance, "ImageWebFetch")
streams_render.display_views(instance, "VideoRcvKafka")
streams_render.display_views(instance, "ImageAnalysis")

## Compose and submit 'ImageWebFetch' 
Fetching from Wiki feed, you will not get anything here unless you have executed the imgAna_3 notebook. 
Disabling the following cell, not deleting since this all should be moving toward 
interesting application. 

Input tuple has URL of the image, fetch the image and publish to 'image_active'.



In [ ]:
%%script false --no-raise-error

import credential
def ImageWebFetch():
    """Fetch image using the url, place encode image in Stream.
    The encoded image is ascii so it can pass through json.
    
    - fetch the image into 'image_string'
    
    """
    topo = Topology("ImageWebFetch")
    topo.add_pip_package('opencv-contrib-python')
    soup_active = topo.subscribe(topic="soup_active")
    active_image = soup_active.map(cvsupport.ImageFetch(), name="image_fetch")
    active_image.view(name="image_fetch", description="encoded binary image")
    active_image.publish(topic="image_active")
    return topo


commonSubmit(cfg, "Streaming3Turbine", ImageWebFetch())    

### View the images fetched by 'ImageWebFetch'

In [ ]:
%%script false --no-raise-error

streams_render.display_views(instance, job_name="ImageWebFetch")

In [ ]:
%%script false --no-raise-error

image_view = instance.get_views(name="image_fetch")[0]
image_view.start_data_fetch()
frame_tuples = image_view.fetch_tuples(max_tuples=30, timeout=30)
print("Number of images:", len(frame_tuples))
image_view.stop_data_fetch()

In [ ]:
%%script false --no-raise-error

## Display the frame_tuples just fetched
image_frame = widgets.Output(layout={'border': '1px solid red','width':'50%','height':'300pt'})
dashboard_frame = widgets.VBox([image_frame])
display(dashboard_frame)

for frame in frame_tuples:
    image_string = frame['image_string']
    with Image.open(io.BytesIO(base64.b64decode(image_string))) as bin_image:
        with image_frame:
            display(bin_image)
            clear_output(wait=True)
            time.sleep(2)

## Compose and submit 'VideoRcvKafka' 
VideSndKafka.py opens video files and sends frames via EventStream/Kafka. The received frames
are published to 'image_active'.



In [ ]:
def VideoRcvKafka():
    """Recieve video frame on topic and publish to 'image_string'. 
    
    Notes:
        - The script VideoSndKafka.py pushed video frame onto the topic.
    
    
    """
    topo = Topology("VideoRcvKafka")
    topo.add_pip_package('opencv-contrib-python')

    video_chunk = eventstreams.subscribe(topo, schema=CommonSchema.Json, 
                                         credentials=json.loads(credential.magsEventStream),
                                         topic='VideoFrame' )
    kafka_frame = video_chunk.map(cvsupport.BuildVideoFrame(), name="kafka_frame")
    kafka_frame.view(name="frame_kafka", description="frame from kafka image")
    kafka_frame.publish(topic="image_active")
    return topo


commonSubmit(cfg, "Streaming3Turbine", VideoRcvKafka())  

### View the frames recieved 'VideoRcvKafka'
The video feed frames are sent by VideoSndKafka.py, via EventStream/Kafka to this application.
The received frames are published.


In [ ]:
streams_render.display_views(instance, job_name="VideoRcvKafka")

In [ ]:
## Fetch some frames from the view
##   The VideoSndKafka script throttles the speed it send out frames.
frame_view = instance.get_views(name="frame_kafka")[0]
frame_view.start_data_fetch()
frame_tuples = frame_view.fetch_tuples(max_tuples=100, timeout=10)
print("Number of frame_tuples:", len(frame_tuples))
frame_view.stop_data_fetch()

In [ ]:
## Display the frame_tuples just fetched
video_frame = widgets.Output(layout={'border': '1px solid red','width':'50%','height':'300pt'})
dashboard_frame = widgets.VBox([video_frame])
display(dashboard_frame)

for frame in frame_tuples:
    image_string = frame['image_string']
    with Image.open(io.BytesIO(base64.b64decode(image_string))) as bin_image:
        with video_frame:
            display(bin_image)
            clear_output(wait=True)
            time.sleep(2)

## Compose and submit 'ImageAnalysis' application
- Two analysis operators thatdo analysis: findFace , findObject.
- Two applications feeding via pub/sub : VideoRcvKafka, ImageWebFetch
- This takes a longgg time to subumit...


In [ ]:
## compose the findImageElements application
def ImageAnalysis():
    """Analyse the images with various tools. 
       - Subscribe to 'image_active' with encoded images in 'image_string'
       - process 'image_string' with FaceRegions into 'face_regions' 
       - process 'image_string' with ObjectRegions into 'object_regions'
       - 
       
    """
    topo = Topology("ImageAnalysis")
    topo.add_file_dependency('../datasets/haarcascade_frontalface_default.xml', 'etc')
    topo.add_file_dependency('../datasets/yolov3.weights', 'etc')
    topo.add_file_dependency('../datasets/yolov3.cfg', 'etc')
    topo.add_pip_package('opencv-contrib-python')
    
    image_active = topo.subscribe(topic="image_active")
    # Find faces analysis ....
    face_regions = image_active.map(cvsupport.FaceRegions(), name="face_regions")
    face_trimmed = face_regions.map(lambda t: {
        'url':t['img_desc'][0]['img'], 
        'face_regions':t['face_regions'],
        'image_string':t['image_string']
    }, name="faces_trimmed")
    face_trimmed.view(name="faces_view", description="faces regions")
    # Find objects analysis ...
    object_regions = image_active.map(cvsupport.ObjectRegions(classes="../datasets/yolov3.txt"), name="object_fetch")
    object_trimmed = object_regions.map(lambda t: {
        'url':t['img_desc'][0]['img'], 
        'object_regions':t['object_regions'],
        'image_string':t['image_string']
    }, name="objects_trimmed")
    object_trimmed.view(name="objects_view", description="object regions")
    return topo


## TODO turn into function - huge duplication 
## Compose and submit the findElements Application 
topo = ImageAnalysis()


commonSubmit(cfg, "Streaming3Turbine", ImageAnalysis())    


# View the ImageAnalysis processing

All the processes 'healthy' ?

In [ ]:
streams_render.list_jobs(instance, cancel=True)

What are the ImageAnalysis views.

In [ ]:
streams_render.display_views(instance, job_name="ImageAnalysis")

## View the ImageAnalysis application's FacesRegions results

In [ ]:
## Fetch FacesRegions start/get/stop
faces_view = instance.get_views(name="faces_view")[0]
faces_view.start_data_fetch()
faces_tuples = faces_view.fetch_tuples(max_tuples=100, timeout=10)
print("Number of faces_tuples:", len(faces_tuples))
faces_view.stop_data_fetch()

In [ ]:
# render the results.... only images with regions will arrive via this view
source_face = widgets.Label(value="Source", layout={'border': '1px solid green','width':'50%'})
rect_face = widgets.Output(layout={'border': '1px solid red','width':'50%','height':'300pt'})
url_face = widgets.Label(value="URL", layout={'border': '1px solid green','width':'50%'})
dashboard_face = widgets.VBox([source_face, rect_face, url_face])
display(dashboard_face)

# Fetch the tuples from the view
count = 0
for face in faces_tuples:
    url_face.value = face['url']
    #source_face.value = trimmed['source']  # TODO - move accros the source.
    source_face.value = str(count)
    count += 1
    image_string = face['image_string']
    with Image.open(io.BytesIO(base64.b64decode(image_string))) as bin_image:
        for rect in face['face_regions']:
            inscribe_rect(bin_image, rect)
        with rect_face:
            display(resize_image(bin_image, baseheight=600))
            clear_output(wait=True)
    time.sleep(2)


## View the ImageAnalysis application's ObjectRegions results

 Collect some images from the view. 

In [ ]:
## Fetch ObjectRegions : start / get / stop
objects_view = instance.get_views(name="objects_view")[0]
objects_view.start_data_fetch()
objects_tuples = objects_view.fetch_tuples(max_tuples=100, timeout=10)
print("Number of object_tuples:", len(objects_tuples))
objects_view.stop_data_fetch()

Display the images and the objects they found. 

In [ ]:
# render the results.... only images with regions will arrive via this view
source_object = widgets.Label(value="Source", layout={'border': '1px solid green','width':'50%'})
rect_object = widgets.Output(layout={'border': '1px solid red','width':'50%','height':'300pt'})
url_object = widgets.Label(value="URL", layout={'border': '1px solid green','width':'50%'})
class_object = widgets.Label(value="CLASS", layout={'border': '1px solid green','width':'50%'})
dashboard_face = widgets.VBox([source_object, rect_object, url_object, class_object])
display(dashboard_face)

# Fetch the tuples from the view
count = 0
for _object in objects_tuples:
    url_object.value = _object['url']
    #source_face.value = trimmed['source']  # TODO - move accros the source.
    source_object.value = str(count)
    count += 1
    image_string = _object['image_string']
    class_string = ""
    with Image.open(io.BytesIO(base64.b64decode(image_string))) as bin_image:
        for rect in _object['object_regions']:
            inscribe_rect(bin_image, rect['region'])
            class_string = ",".join([class_string,rect['class']])
            class_object.value = class_string
        with rect_object:
            display(resize_image(bin_image, baseheight=600))
            clear_output(wait=True)
    time.sleep(2)
